In [ ]:
import time
from collections import defaultdict

import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
import pandas as pd
from brian2 import *
from smallworld import get_smallworld_graph
from smallworld.draw import draw_network
from tqdm import tqdm

import random
%matplotlib inline

In [ ]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), "ok", ms=10)
    plot(ones(Nt), arange(Nt), "ok", ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], "-k")
    xticks([0, 1], ["Source", "Target"])
    ylabel("Neuron index")
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, "ok")
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel("Source neuron index")
    ylabel("Target neuron index")

In [ ]:
time0 = time.time()
# イジケビッチニューロンの定義
eqs = Equations(
    """
dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u+I : volt
du/dt = a*(b*v-u)                                : volt/second
I                                                : volt/second
a                                                : 1/second
b                                                : 1/second
c                                                : volt
d                                                : volt/second
"""
)

reset = """
v = c
u = u + d
"""


taupre = taupost = 20 * ms
# 論文の設定
wmax = 10  # 本当に(ry
Apre = 0.1  # 本当にvolt?
Apost = -0.12  # 本当にvolt?

n = 1000  # number of neurons in one group
neuron_group_count = 100
R = 0.8  # ratio about excitory-inhibitory neurons

# 各ニューロングループの生成
# 30mvでスパイクが発生する。数値積分法はeuler
P = NeuronGroup(n * neuron_group_count, model=eqs, threshold="v>30*mvolt", reset=reset, method="euler")
re = np.random.random(int(n * R))
ri = np.random.random(round(n * (1 - R)))



groups = []
# サブグループに分ける
for i in tqdm(range(neuron_group_count)):
    start = int(i * n)
    end = int((i + 1) * n)

    group = P[start:end]
    # 興奮性
    Pe = group[: int(n * R)]
    # 抑制性
    Pi = group[int(n * R) :]

    # 各種設定
    Pe.a = 0.02 / msecond # 正
    Pe.b = 0.2 / msecond # 正
    Pe.c = (15 * re ** 2 - 65) * mvolt
    Pe.d = (-6 * re ** 2 + 8) * mvolt / msecond
    Pe.I = 20 * mvolt / msecond # 他が全部mVなのにこれだけvoltはおかしい /msecondじゃないと頻度が低すぎる
    Pe.u = Pe.b*Pe.c
    Pe.v = Pe.c

    Pi.a = (0.08 * ri ** 2 + 0.02) * 1 / msecond # 正
    Pi.b = (-0.05 * ri ** 2 + 0.25) * 1 / msecond # 正
    Pi.c = -65 * mvolt
    Pi.d = 2 * mvolt / msecond
    Pi.I = 20 * mvolt / msecond
    Pi.u = Pi.b*Pi.c
    Pi.v = Pi.c
    
    # グループ内の接続
    # 興奮性ニューロン to 同じニューロングループ内の100個のニューロンへのランダム接続
    # Ce = Synapses(Pe,group,on_pre='v_post +=6 * mV')
    Ce = Synapses(
        Pe,
        group,
        """
        w : 1
        dapre/dt = -apre/taupre : 1 (event-driven)
        dapost/dt = -apost/taupost : 1 (event-driven)
        """,
        on_pre="""
        v_post += w * mV
        apre += Apre
        w = clip(w+apost, 0, wmax)
        """,
        on_post="""
        apost += Apost
        w = clip(w+apre, 0, wmax)
        """,
    )
    Ce.connect(p=0.1)
    Ce.w = 6.0
    Ce.delay = round(random.uniform(0, 20), 2) * ms
    # 抑制性ニューロン　to 同じニューロングループ内の100個の興奮性ニューロンへのランダム接続
    # Ci = Synapses(Pi,Pe,on_pre='v_post +=-5*mV')
    Ci = Synapses(
        Pi,
        Pe,
        """
        w : 1
        dapre/dt = -apre/taupre : 1 (event-driven)
        dapost/dt = -apost/taupost : 1 (event-driven)
        """,
        on_pre="""
        v_post += w * mV
        apre += Apre
        w = clip(w+apost, -wmax, 0)
        """,
        on_post="""
        apost += Apost
        w = clip(w+apre, -wmax, 0)
        """,
    )
    Ci.connect(p=0.125)
    Ci.w = -5.0
    Ci.delay = 1 * ms

    groups.append((group, Pe, Pi, Ce, Ci))

time1 = time.time()
print("グループ内の配線をするまでにかかった時間", time1 - time0, "sec")

In [ ]:
# WSモデルに従い、各グループの興奮性ニューロンから隣接する6つのノードへの接続と再配線を行う

# define network parameters
N = neuron_group_count
k_over_2 = 3
# k_over_2 = 1
beta = 1.0
label = r"$\beta=0$"

focal_node = 0
# generate small-world graphs and draw
G = get_smallworld_graph(N, k_over_2, beta)
inter_synapses = []

for edge in tqdm(list(G.edges())):
    source_group_Pe = groups[edge[0]][1]
    target_group = groups[edge[1]][0]

    Ce = Synapses(
        source_group_Pe,
        target_group,
        """
        w : 1
        dapre/dt = -apre/taupre : 1 (event-driven)
        dapost/dt = -apost/taupost : 1 (event-driven)
        """,
        on_pre="""
        v_post += w *mV
        apre += Apre
        w = clip(w+apost, 0, wmax)
        """,
        on_post="""
        apost += Apost
        w = clip(w+apre, 0, wmax)
        """,
    )
    # 各興奮性ニューロンが、他のニューロングループと三本の接続を持つので、接続する確率は3/1000
    Ce.connect(p=0.003)
    Ce.w = 6.0
    Ce.delay = round(random.uniform(10, 30), 2) * ms
    inter_synapses.append(Ce)

time2 = time.time()
print("グループ外の配線をするまでにかかった時間", time2 - time1, "sec")

In [ ]:
net = Network([group[3] for group in groups],[group[4] for group in groups],inter_synapses,P)
# net = Network([group[3] for group in groups],[group[4] for group in groups],P)
# net = Network([group[3] for group in groups],P)

# とりあええず1000秒動かす
run_time_ms = 100 * 1000
# run_time_ms = 1000
# run_time_ms = 10
value_interval_ms = 1
defaultclock.dt = value_interval_ms * ms

# V = StateMonitor(P, "v", record=True)
# A = StateMonitor(P, "a", record=True)
# B = StateMonitor(P, "b", record=True)
# C = StateMonitor(P, "c", record=True)
# D = StateMonitor(P, "d", record=True)
# U = StateMonitor(P, "u", record=True)
# S = SpikeMonitor(P)

# net.add(V)
# net.add(A)
# net.add(B)
# net.add(C)
# net.add(D)
# net.add(U)
# net.add(S)

net.run(run_time_ms * ms,report='stdout')

time3 = time.time()
print("最初の千秒までにかかった時間", time3 - time2, "sec")

In [ ]:
# STDPの設定を外す
for group in groups:
    group[3].pre.code = "v_post +=w* mV"
    group[3].post.code = ""
    group[4].pre.code = "v_post +=w *mV"
    group[4].post.code = ""
    
for inter in inter_synapses:
    inter.pre.code = "v_post +=w *mV"
    inter.post.code = ""

run_time_ms = 100 * 1000

# 100秒動かす
net.run(run_time_ms * ms,report='stdout')

time4 = time.time()
print("次の100秒までにかかった時間", time4 - time3, "sec")

In [ ]:
# inputの設定を外す
new_I = array([0.0 for i in range(n * neuron_group_count)])
P.I = new_I * volt / second

V = StateMonitor(P, "v", record=True)
A = StateMonitor(P, "a", record=True)
B = StateMonitor(P, "b", record=True)
C = StateMonitor(P, "c", record=True)
D = StateMonitor(P, "d", record=True)
U = StateMonitor(P, "u", record=True)
S = SpikeMonitor(P)

net.add(V)
net.add(A)
net.add(B)
net.add(C)
net.add(D)
net.add(U)
net.add(S)

# 100秒動かす
run_time_ms = 100 * 1000
# value_interval_ms = 1000
# defaultclock.dt = value_interval_ms * ms

# V = StateMonitor(P, "v", record=True)
# net.add(V)
net.run(run_time_ms * ms,report='stdout')

time5 = time.time()
print("次の100秒までにかかった時間", time5 - time4, "sec")

In [ ]:
lap = defaultdict(list)
for i in tqdm(range(neuron_group_count)):
    start = int(i * n)
    end = int((i + 1) * n)

    group = V.v[start:end]/mV
    # 興奮性
    Pe = group[: int(n * R)]
    lap[i] = np.mean(np.array(Pe), axis=0)

time6 = time.time()
print("lap計算までにかかった時間", time6 - time5, "sec")

pd.DataFrame(lap[0]).to_csv("test_before.csv", index=False)

In [ ]:
spikes[0][0]

In [ ]:
np.array(V.v).shape

In [ ]:
np.array(V.v[0][0:100])

In [ ]:
# 全てのニューロングループに対してMSEの計算を行う
results = []
for i in range(neuron_group_count):
    result = nk.entropy_multiscale(signal=np.array(lap[i]), scale=40, dimension=1)
    results.append(result[1]["Values"])

time7 = time.time()
print("mse計算までにかかった時間", time7 - time6, "sec")

pd.DataFrame(results).to_csv("test.csv", index=False)
time8 = time.time()
print("全体の処理時間", time8 - time0, "sec")

In [ ]:
%matplotlib inline
# visualise_connectivity(groups[0][3])
# visualise_connectivity(groups[0][4])
# visualise_connectivity(groups[1][3])

In [ ]:
# np.array(S.s)[0]

In [ ]:
# np.array(V.v[0]/mV)
# P.a[0]
# print(np.array(P.u[0]))
# print(U.u[0])
# P.d[0]
# np.array(V.v[0]/mV)>30 == True
# len(V.v[0])
# set(np.array(P.w[0]))
# print(set(np.array(groups[0][3].w)))
# print(set(np.array(groups[0][4].w)))

In [ ]:
# spikes[0]

In [ ]:
%matplotlib inline
plt.plot(V.t[:100]/ms,V.v[0][:100]/mV)
spikes = S.spike_trains()
# for s in spikes[0]:
    # axvline(s/ms,ls='--',c='C1',lw=3)
xlabel('Time (ms)')
ylabel('mV')

In [ ]:
# np.array(V.v[0][4]/mV)
np.array(V.v[0][0:100]/mV)

In [ ]:
%matplotlib inline
plt.plot(V.t/ms,V.v[801]/mV)
spikes = S.spike_trains()
for s in spikes[0]:
    axvline(s/ms,ls='--',c='C1',lw=3)
xlabel('Time (ms)')
ylabel('mV')

In [ ]:
%matplotlib inline
plt.plot(A.t/ms,A.a[0])
xlabel('Time (ms)')

In [ ]:
%matplotlib inline
plt.plot(B.t/ms,B.b[0])
xlabel('Time (ms)')

In [ ]:
%matplotlib inline
plt.plot(C.t/ms,C.c[0]/mV)
xlabel('Time (ms)')
ylabel('mV')

In [ ]:
%matplotlib inline
plt.plot(D.t/ms,D.d[0]/mV)
xlabel('Time (ms)')
ylabel('mV')

In [ ]:
%matplotlib inline
plt.plot(U.t/ms,U.u[0])
xlabel('Time (ms)')

In [ ]:
%matplotlib inline
plt.plot(D.t/ms,D.d[810])
xlabel('Time (ms)')
ylabel('mV')

In [ ]:
%matplotlib inline
plt.plot(U.t/ms,U.u[840])
xlabel('Time (ms)')
ylabel('mV')